Task 4: Bucket FICO scores

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Task 3 and 4_Loan_Data.csv to Task 3 and 4_Loan_Data.csv


In [4]:
import pandas as pd
import numpy as np
from math import log

# Load the dataset
df = pd.read_csv("Task 3 and 4_Loan_Data.csv")

# Extract necessary columns
x = df['default'].to_list()  # Default status
y = df['fico_score'].to_list()  # FICO scores
n = len(x)
print(f"Number of records: {len(x)}, Number of FICO scores: {len(y)}")

# Initialize arrays to store defaults and total counts for each FICO score
default = [0 for _ in range(851)]
total = [0 for _ in range(851)]

# Count the number of defaults and totals for each FICO score
for i in range(n):
    y[i] = int(y[i])
    default[y[i] - 300] += x[i]  # FICO scores are adjusted by 300 because they start from 300
    total[y[i] - 300] += 1

# Cumulative sums for defaults and totals
for i in range(1, 551):
    default[i] += default[i - 1]
    total[i] += total[i - 1]

# Log-likelihood function
def log_likelihood(n, k):
    p = k / n
    if p == 0 or p == 1:
        return 0
    return k * log(p) + (n - k) * log(1 - p)

# Number of buckets
r = 10

# Dynamic programming table
dp = [[[-10**18, 0] for _ in range(551)] for _ in range(r + 1)]

# Dynamic programming to calculate log-likelihood and track optimal splits
for i in range(r + 1):
    for j in range(551):
        if i == 0:
            dp[i][j][0] = 0  # Base case, no buckets
        else:
            for k in range(j):
                if total[j] == total[k]:
                    continue
                if i == 1:
                    dp[i][j][0] = log_likelihood(total[j], default[j])
                else:
                    new_ll = log_likelihood(total[j] - total[k], default[j] - default[k]) + dp[i - 1][k][0]
                    if dp[i][j][0] < new_ll:
                        dp[i][j][0] = new_ll
                        dp[i][j][1] = k

# Final log-likelihood value
print(f"Final Log-Likelihood: {round(dp[r][550][0], 4)}")

# Trace back to find the optimal FICO score splits (bucket boundaries)
k = 550
buckets = []
while r > 0:
    buckets.append(k + 300)  # Convert back to original FICO score by adding 300
    k = dp[r][k][1]
    r -= 1

buckets.reverse()  # Reverse to get buckets in ascending order
print(f"Optimal FICO Score Buckets: {buckets}")


Number of records: 10000, Number of FICO scores: 10000
Final Log-Likelihood: -4217.8245
Optimal FICO Score Buckets: [520, 552, 580, 611, 649, 696, 732, 752, 753, 850]
